# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat publications.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url
2020-05-20	Variability in the analysis of a single neuroimaging dataset by many teams	Nature	Data analysis workflows in many scientific domains have become increasingly complex and flexible. Here we assess the effect of this flexibility on the results of functional magnetic resonance imaging by asking 70 independent teams to analyse the same dataset, testing the same 9 ex-ante hypotheses1. The flexibility of analytical approaches is exemplified by the fact that no two teams chose identical workflows to analyse the data. This flexibility resulted in sizeable variation in the results of hypothesis tests, even for teams whose statistical maps were highly correlated at intermediate stages of the analysis pipeline. Variation in reported results was related to several aspects of analysis methodology. Notably, a meta-analytical approach that aggregated information across teams yielded a significant consensus in activated regions. Furthe

## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,2020-05-20,Variability in the analysis of a single neuroi...,Nature,Data analysis workflows in many scientific dom...,"Botvinik-Nezer, R., Holzmeister, F., Camerer, ...",Variability in the analysis,http://academicpages.github.io/files/paper1.pdf
1,2020-10-29,The Role of Social Reward and Corticostriatal ...,J. Psychiatric Brain Sci.,This report describes an ongoing R03 grant tha...,"Sazhin, D., Frazier, A. M., Haynes, C. R., Joh...",The Role of Social Reward ...in Substance Use,http://academicpages.github.io/files/paper2.pdf
2,2021-05-01,Inflammatory Signaling and Corticostriatal Fun...,Biological Psychiatry,Growing research suggests that low-grade infla...,"Chat, I. K. Y., Bart, C., Dennison, J., Smith,...",Inflammatory Signaling,http://academicpages.github.io/files/paper3.pdf
3,2022-02-08,Decision neuroscience and neuroeconomics: Rece...,WIREs Cognitive Science,"In the past decade, decision neuroscience and ...","Dennison, J. B., Sazhin, D., & Smith, D. V. (2...",Decision neuroscience and neuroeconomics:,http://academicpages.github.io/files/paper4.pdf
4,2023-01-19,Substance use is related to differential activ...,BioRxiv,Substance use (SU) has been linked to alterati...,"Wyngaarden III, J. B., Johnston, C. R., Sazhin...",Substance use is related to differential activ...,http://academicpages.github.io/files/paper5.pdf
5,2023-02-07,Tensorial Independent Component Analysis Revea...,BioRxiv,Major depressive disorder (MDD) has been assoc...,"Dennison, J. B., Tepfer, L. J., & Smith, D. V....",Tensorial Independent Component Analysis Reveals,http://academicpages.github.io/files/paper6.pdf


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [6]:
!ls ../_publications/

 2009-10-01-paper-title-number-1.md
 2010-10-01-paper-title-number-2.md
 2015-10-01-paper-title-number-3.md
'2020-05-20-Variability in the analysis.md'
'2020-10-29-The Role of Social Reward ...in Substance Use.md'
'2021-05-01-Inflammatory Signaling.md'
'2022-02-08-Decision neuroscience and neuroeconomics:.md'
'2023-01-19-Substance use is related to differential activation and connectivity.md'
'2023-02-07-Tensorial Independent Component Analysis Reveals .md'


In [7]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

---
title: "Variability in the analysis of a single neuroimaging dataset by many teams"
collection: publications
permalink: /publication/2009-10-01-paper-title-number-1
excerpt: 'This paper is about the number 1. The number 2 is left for future work.'
date: 2020-05-20
venue: 'Nature'
paperurl: 'http://jbdenniso.github.io/files/Nature-20_Variability.pdf'
citation: 'Botvinik-Nezer, R., Holzmeister, F., Camerer, C. F., Dreber, A., Huber, J., Johannesson, M., ... & Rieck, J. R. (2020). &quot;Variability in the analysis of a single neuroimaging dataset by many teams.&quot; <i>Nature </i>, 582(7810), 84-88.'
---
Data analysis workflows in many scientific domains have become increasingly complex and flexible. Here we assess the effect of this flexibility on the results of functional magnetic resonance imaging by asking 70 independent teams to analyse the same dataset, testing the same 9 ex-ante hypotheses1. The flexibility of analytical approaches is exemplified by the fact that no two teams 